In [1]:
import csv
import pandas as pd
from fastai.text.all import *
import torch
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import json

/home/karynaur/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
a=['1','2',['1','2'],['1','2']]
b=json.dumps(a)
c=json.loads(b)
with open('test.json', 'w') as json_file:
    json.dump(b, json_file)

In [3]:
with open('data/genius.csv',encoding="utf8",errors='ignore') as f:
    df=pd.read_csv(f)
df.keys()

Index(['Artist', 'Song', 'Lyrics'], dtype='object')

In [4]:
df.shape

(845, 3)

In [5]:
pretrained_weights = 'gpt2'
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)

In [6]:
def tokenize(text):
    toks = tokenizer.tokenize(text)
    return tensor(tokenizer.convert_tokens_to_ids(toks))



class TransformersTokenizer(Transform):
    def __init__(self, tokenizer): self.tokenizer = tokenizer
    def encodes(self, x): 
        return x if isinstance(x, Tensor) else tokenize(x)
        
    def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

class DropOutput(Callback):
    def after_pred(self):self.learn.pred=self.pred[0]


In [7]:
artists={'Justin Bieber':'beiber',
        'Adele':'adele',
        'Drake':'drake',
        'The Weeknd':'weeknd',
        'Ariana Grande':'ariana',
        'Ed Sheeran':'edsheeran',
        'Coldplay':'coldplay',
        'Miley Cyrus':'miley',
        'Backstreet Boys':'backstreet',
        'Eminem':'eminem',
        '21 Savage':'21savage',
        'Linkin Park':'linkinpark',
        '\u200btwenty one pilots':'21pilots',
        'Kanye West':'kanye',
        'Taylor Swift':'taylor'}

In [55]:
dic={
    'Justin Bieber':['Love Yourself','Sorry','Baby','Purpose','Boyfriend','Nothing Like Us','Never Say Never'],
    'Adele':['Hello','Someone Like You','All I Ask','When We Were Young','Rolling in the Deep','Skyfall','Daydreamer'] ,
    'Drake':['God’s Plan','In My Feelings','Hotline Bling','One Dance','Nice For What','Hold On, We’re Going Home','Die In Your Arms'] ,
    'The Weeknd':['Starboy','Can’t Feel My Face','Call Out My Name','Reminder','Often','Feel It Coming','Wicked Games'] ,
    'Ariana Grande':['thank u, next','7 rings','God is a woman','no tears left to cry','Dangerous Woman','breathin','Bad Decisions'] ,
    'Ed Sheeran':['Shape of You','Perfect','Castle on the Hill','Happier','Photograph','Galway Girl','Nina'] ,
    'Coldplay':['The Scientist','Viva La Vida','Kaleidoscope','Yellow','Hymn for the Weekend','A Sky Full of Stars','Hypnotised'] ,
    'Miley Cyrus':['Malibu','Wrecking Ball','We Can’t Stop','Slide Away','Mother’s Daughter','Adore You','Party in the U.S.A.'] ,
    'Backstreet Boys':['I Want It That Way','Don’t Go Breaking My Heart','As Long As You Love Me','Everybody (Backstreet’s Back)','I’ll Never Break Your Heart','What Makes You Different (Makes You Beautiful)','OK'] ,
    'Eminem':['Rap God','The Monster','Lose Yourself','Lucky You','Stan','Love the Way You Lie','The Real Slim Shady'] ,
    '21 Savage':['Bank Account','a lot','Red Opps','ball w/o you','can’t leave without it','monster','Numb'] ,
    'Linkin Park':['Heavy','In the End','One More Light','Numb','Battle Symphony','Good Goodbye','CASTLE OF GLASS'] ,
    '\u200btwenty one pilots':['Heathens','Stressed Out','Ride','Chlorine','Nico and the Niners','Car Radio','Tear in My Heart'] ,
    'Kanye West':['Mercy','Father Stretch My Hands, Pt. 1','Ultralight Beam','Monster','West Bound 2','New Slaves','Blood on the Leaves'],
    'Taylor Swift':['Shake It Off','Blank Space','Bad Blood','Look What You Made Me Do','Gorgeous','Style']
}

In [95]:
sng=[]
for i in dic:
    for k in dic[i]:
        sng.append(k)


'Sorry'

In [107]:
with open('data/genius.csv',encoding="utf8") as f:
    reader = csv.reader(f)
    dataset = list(reader)
data=list(dataset)
data[7]

['Justin Bieber',
 'I’ll Show You',
 "My life is a movie and everyone's watching\nSo let's get to the good part and past all the nonsense\n\nSometimes it's hard to do the right thing\nWhen the pressure's coming down like lightning\nIt's like they want me to be perfect\nWhen they don't even know that I'm hurting\n\nCause life's not easy, I'm not made out of steel\nDon't forget that I'm human, don't forget that I'm real\nAct like you know me, but you never will\nThere's one thing that I know for sure\nI'll show you\nI'll show you\nI'll show you\nI'll show you\n\nI've got to learn things, learn them the hard way\nGot to see what it feels like, no matter what they say\n\nSometimes it's hard to do the right thing\nWhen the pressure's coming down like lightning\nIt's like they want me to be perfect\nWhen they don't even know that I'm hurting\n\nCause life's not easy, I'm not made out of steel\nDon't forget that I'm human, don't forget that I'm real\nAct like you know me, but you never will\n

In [16]:

def batch(lyrics):
    prompt=''
    if lyrics[0]=='\n':
        count=-1
    else: count=0
    for i in range(len(lyrics)):
        prompt+=lyrics[i]
        if lyrics[i-1]=='\n' and lyrics[i]=='\n':
            
            continue
        if lyrics[i]=='\n':
            count+=1
        if count==4:
            break
    return prompt.replace('\n\n','\n')

In [17]:
def countns(lyrics):
    count=0
    for i in range(len(lyrics)):
        if lyrics[i]=='\n':
            count+=1
    return count
def linebreak(lyrics):
    for i in range(len(lyrics)):
        if lyrics[i]=='\n':
            return i+1
        
countns('asdasd\n\n\n')

3

In [44]:
def extract_prompt(lyrics):
    if lyrics[0]=='\n':
        count=-1
    else: count=0
    for i in range(len(lyrics)):
        if lyrics[i]=='\n':
            count+=1
        if count==2:break
    return i+1
  
def extract_preds(lyrics):
    if lyrics[0]=='\n':
        count=-1
    else: count=0
    for i in range(len(lyrics)):
        if lyrics[i-1]=='\n' and lyrics[i]=='\n':
            
            continue
        if lyrics[i]=='\n':
            count+=1
        if count==5:break
    return i+1


  

In [96]:
data[0]

['Artist', 'Song', 'Lyrics']

In [1]:
dataset=[]
for k in data[1:]:
    if k[1] in sng:
        dataset.append(k)
        prompts=[]
        generate=[]
        n=countns(k[2])
        string=k[2]
        learn=load_learner('models/{}.pkl'.format(artists[k[0]]))
        print('models/{}.pkl'.format(artists[k[0]]))
        for j in range(n-4):
            try:
                p=batch(string)
                br=linebreak(p)
                string=string[br:]

                pr=p[:extract_prompt(p)]
                prompt_ids = tokenizer.encode(pr)
                inp = tensor(prompt_ids)[None]
                count=0
                while(True):
                    preds=learn.model.generate(inp,
                                 do_sample=True, 
                                 max_length=65, 
                                 min_length=5,
                                 top_k=40,
                                 num_return_sequences=1)

                    g=tokenizer.decode(preds[0].cpu().tolist())
                    g=g[:extract_preds(g)].replace("\"\"","").replace("\"","").replace("\n\n","\n").replace("\n\n","\n")
                    count+=1
                    if countns(g)>=4 or count==10:break


                generate.append(g)
                prompts.append(p)

            except IndexError:break
        dataset.pop()
        dataset.append(prompts)
        dataset.append(generate)
with open('lyrics.json', 'w') as json_file:
    json.dump(data, json_file)

NameError: name 'data' is not defined

In [104]:
dataset=[]
for k in data[101:103]:
    dataset.append(k)
dataset

[['Drake',
  'Hotline Bling',
  "You used to call me on my\nYou used to, you used to\nYeah\n\nYou used to call me on my cell phone\nLate night when you need my love\nCall me on my cell phone\nLate night when you need my love\nAnd I know when that hotline bling\nThat can only mean one thing\nI know when that hotline bling\nThat can only mean one thing\n\nEver since I left the city, you\nGot a reputation for yourself now\nEverybody knows and I feel left out\nGirl, you got me down, you got me stressed out\n‘Cause ever since I left the city, you\nStarted wearing less and goin' out more\nGlasses of champagne out on the dance floor\nHangin' with some girls I've never seen before\n\nYou used to call me on my cell phone\nLate night when you need my love\nCall me on my cell phone\nLate night when you need my love\nI know when that hotline bling\nThat can only mean one thing\nI know when that hotline bling\nThat can only mean one thing\n\nEver since I left the city, you, you, you\nYou and me, we